# Robins-Monro algorithm

### Motivating example: Mean estimation

#### 1. 两种空间复杂度$O(1)$的均值计算方法

- 目标：计算$\frac{1}{n}\sum_{i=1}^nx_i$

- 方法一：
$$\begin{align}
init: & k = 0 \\
iterate: &\\
& acc = acc + x_k \\
& k = k + 1 \\
& avg = acc / k
\end{align}$$

- 方法二：
$$\begin{align}
init: & k = 0, avg=x_0 \\
iterate: &\\
& k = k + 1 \\
& avg = avg + \frac{1}{k}(x_{k-1}-avg)  \\
\end{align}$$

- 方法二可以写成另一种等价形式：[为了与math of RL书上的表述一致]
$$\begin{align}
init: & k = 1, w_1=x_1 \\
iterate: &\\
& w_{k+1} = w_k - \frac{1}{k}(w_k-x_{k})  \\
& k = k + 1 \\
\end{align}$$
  - 迭代过程：
    - <img src='pics/mean_accumulation.png' width='50%'>

#### 2. 一种泛化的均值迭代计算方法

- 前述方法二可以归纳为一个迭代表达式：
$$w_{k+1} = w_k - \frac{1}{k}(w_k-x_{k})$$
- 该迭代表达式可以进一步一般化为：
$$w_{k+1} = w_k - \alpha_k(w_k-x_{k})$$
  - 当$\alpha_k$满足一定条件时，可以证明当$k\rightarrow \infty时，w_k \rightarrow E(X)$

## I. RM算法

### I.1 stochastic approximation

- RM算法是stochastic approximation算法中的一种。
- <font color=blue>**stochastic approximation算法**</font>：是一类通过使用迭代法求解优化问题和求根问题的方法。和其他解决优化和求根问题的方法，如梯度下降法不同，stochastic approximation算法的优点在于不需要知道目标函数的或者其梯度的表达式，只需要有足够的函数样本就能使用。
  - <font color=deeppink>**优化问题在一定条件下可以转换为求根问题**</font>：
    - 如果函数是凸函数，那么求最小值的优化问题就是求解$\nabla f(x)=0$的根。

### I.2 RM算法内容

- 问题：求$w^*$，使其满足$g(w^*)=0$，g(w)的表达式未知，但是有很多含噪声的{w，g(w)}样本pairs。
   - 比如一个NN，没有对应的方程表达式，但是有输入和输出样本。

- 方法：

$$迭代求解：w_{k+1}=w_{k} - a_k * \tilde g(w_k,\ \eta_k)，k=1, 2, 3, ...$$
- 说明：
  1. $w_k$是$g(w)=0$函数根的第k次估计
  2. $\tilde g(w_k,\eta_k)是$g(w)$的第k个样本观测值$。$\tilde g(w_k, \eta_k)  = g(w_k) + \eta$
  3. $a_k$是一个positive coefficient

### I.3 RM算法收敛性：almost surely convergence(strong consistent)

- 注意两个概念：
  1. strong consistent = almost surely convergence
  2. weak consistent = converges in probability

- 下面三个条件满足时，$w_k$迭代<font color=green>**almost sure**</font>收敛到$w^*$ ：
  1. <font color=norange>$\mathbf{0<c_1\le \nabla _wg(w)\le c_2}$</font>
  2. <font color=norange>$\mathbf{\sum^{\infty}_{k=1}a_k=\infty,且\sum^{\infty}_{k=1}a_k^2<\infty}$</font>
  3. <font color=norange>$\mathbf{E(\eta_k|W_k)=0, E(\eta^2_k|W_k)<\infty$，其中，$W_k={w_k, w_{k-1}, ..., }}$</font>
     - 当$\eta_k与W_k$无关时，第三个条件简化为$E(\eta_k)=0, E(\eta^2_k)<\infty$

- **条件1的说明**：<font color=norange>$\mathbf{0<c_1\le \nabla _wg(w)\le c_2}$</font>
     - <font color=orange>下限不能取0是保证g(w)=0必然有解的关键。</font>因为此时函数g(w)单调递增，有且只有一个0解。
     - 上限的存在确保了单次迭代后w的小幅变化不会让函数值f(w)发生巨大的变化，导致f(w_k)在0值上下振动。

- **条件2的说明**：<font color=norange>$\mathbf{\sum^{\infty}_{k=1}a_k=\infty,且\sum^{\infty}_{k=1}a_k^2<\infty}$</font>
     - $\sum^{\infty}_{k=1}a_k=\infty$：确保遍历范围达到所有可能的取值，不管初始$w_0$距离$w^*$有多远，都能收敛到最优值。
       - 说明：
         - $w_1-w_{\infty}=\sum_{k=1}^{\infty}a_k*\tilde g(w_k, \eta_k)$，该式左边表示整个迭代过程中w所能覆盖的最远范围。如果没有$\sum_{k=1}^{\infty}a_k\rightarrow \infty$这个条件，上式右边值有限，就意味着遍历所能达到的最远范围有限。
     - $\sum^{\infty}_{k=1}a_k^2<\infty意味着当k\rightarrow \infty时，\alpha_k\rightarrow 0$：确保当迭代次数足够大，到达最优值附近的时候，$w_k$不要大幅抖动。
       - 说明：
         - $w_{k+1}-w_k=-\alpha_k * \tilde g(w_k,\ \eta_k)$，如果$当k\rightarrow \infty时，\alpha_k\rightarrow 0$不成立，则$w_{k+1}-w_k$不会逼近0，也就是说，$w_k$在迭代次数达到无穷大后，仍然会振动。

- 实践：
  - 满足上述条件的典型例子：$a_k=\frac{1}{k}$。<font color=orange>虽然满足上面两个条件，但也有明显的缺点。当迭代次数k越来越大以后，$\frac{1}{k}$逼近0的速度很快，导致后面的样本对w更新的影响很小，所以不常用。</font>
  - 实践中常见取$\alpha_k$为一个很小的常数，此时条件2中$\sum^{\infty}_{k=1}a_k^2<\infty$不满足，但是如果允许一定的最小误差，影响不大。

### I.4 无偏性分析

- <font color=red>**RM算法的收敛能保证convergence，但不能保证unbias。**</font>

#### I.4.1 概念: asymptotically unbiased

- **定义**：
  - 当如果估计量样本量趋于无穷大时是无偏估计，就称该估计量是asymptotically unbiased
$$\lim_{n \to \infty} E\hat \theta_n = \theta$$ 

- <font color=blue>consistent是比asymptotically unbiased更强的性质。asymptotically unbiased的估计量不一定是consistent的。但consistent估计量一定是asymptotically unbiased。</font>
  - 比如：i.i.d.样本{x1,x2,...,xn},EX=μ,Var(X)=σ2.如果用x1+1/n(注意不是xi,是第1个样本x1)作为μ的估计量，那么该估计量有bias 1/n，但是满足asymptotically unbiased

#### I.4.2 在求函数期望值$Ef(x)$时，RM算法得到的估计量满足asymptotically unbiased

- 迭代式：$\theta_{k+1}=\theta_k -\alpha_k(\theta_k - f(x_k))$
- 无偏性证明：
$$\begin{align}
\alpha_k & = \frac{E\theta_{k+1}-E\theta_k}{Ef(x_k)-E\theta _k}  = \frac{E\theta_{k+1}-E\theta_k}{\mu-E\theta _k}\\
1- \alpha_k & = 1- \frac{E\theta_{k+1}-E\theta_k}{\mu-E\theta _k}=\frac{\mu-E\theta_{k+1}}{\mu-E\theta _k}\\
\prod_{i=1}^k (1-\alpha_i)& = \frac{\mu-E\theta_{k+1}}{\mu-E\theta _1}\\
&\because \lim_{k \to \infty} \prod_{i=1}^k (1-\alpha_i) = 0, then\ \lim_{k \to \infty} (\mu-E\theta_{k+1})=0
\end{align}$$
  - <font color=green>这里无偏性的分析是因为Ef(x)为目标时构造的$\bar g(\theta, \eta)$正好可以分解成$\theta - f(x)$的形式。但一般化的RM算法中，这个特征并不普遍存在，也因此通常没有这里的无偏性质。</font>

- <font color=blue>**收敛过程中的bias特征：随着迭代次数的增加，C会越来越小，此时bias也会越来越小**</font>
  - 分析：
$$\begin{align}
&取C=\prod_{i=1}^k (1-\alpha_i),有：\\
&\mu-E\theta_{k+1} = (\mu-E\theta _1)\prod_{i=1}^k (1-\alpha_i)= C(\mu-E\theta _1)\\
\end{align}$$
    - 因为 $1-\alpha_k<1$，所以随着迭代次数的增加，C会越来越小，此时bias也会越来越小。

#### I.4.3 RM算法得到的估计量is biased

- <font color=brown>即使在特定场景下，如求$Ef(x)$时可以证明RM迭代得到的估计量是asymptotically unbiased。但其他分析中，上述性质并不普遍存在。RM算法估计量通常情况下是有偏的。</font>

### I.5 算法的intuition

- 当满足收敛的三个条件时，函数图形可以简单如下图所示：
  - <img src='pics/rm_intuition.png' width='40%'>

- 简化假设样本中的噪音较小，接近于0，有：
  - $w_k > w^*时，g(w)>0，w_{k+1}=w_k-\alpha_k * g(w_k) < w_k, 只要\alpha_k * g(w_k)足够小，则w^*<w_{k+1}<w_k，w_{k+1}比w_k更接近w^*$
  - $w_k < w^*时，g(w)<0，w_{k+1}=w_k-\alpha_k * g(w_k) > w_k, 同样，w_{k+1}比w_k更接近w^*$

### I.6 RM算法收敛性的证明

#### I.6.1 Dvorotzky's theorem

- 随机过程:
$$\Delta_{k+1}=(1-\alpha_k)*\Delta_k+\beta_k*\eta_k$$
  - 其中$\{\alpha_k\}_{k=1}^{\infty}, \{\beta_k\}_{k=1}^{\infty}, \{\eta_k\}_{k=1}^{\infty}$都是随机序列，且$\alpha_k \ge 0, \beta_k \ge 0$。
  - 在满足以下条件时，$\Delta_k$ almost surely converge to zero:
    1. $\sum^{\infty}_{k=1}a_k=\infty,\ \sum^{\infty}_{k=1}a_k^2<\infty,且\sum^{\infty}_{k=1}\beta _k^2<\infty$ uniformly almost surely
    2. $E(\eta_k|H_k)=0, E(\eta^2_k|H_k)<C$ almost surely.$H_k = \{\Delta_k,\Delta_{k-1}, ..., \alpha_k, \alpha_{k-1},...,\beta_k, \beta_{k-1} \}$
- <font color=brown>[证明见math of RL,6.3.1节 page119]</font>

#### I.6.2 证明RM算法满足Dvorotzky's theorem

- 简单分析，详细条件见math of RL,6.3.3节
$$\begin{align}
w_{k+1} & = w_{k} - a_k * \tilde g(w_k,\ \eta_k) \\
& = w_{k} - a_k * [g(w_k)+\eta_k] \\
w_{k+1} - w^* & = w_{k} - w^* - a_k * [g(w_k)+\eta_k] \\
& =  w_{k} - w^* - a_k * [g(w_k) - g(w^*)+\eta_k] \\
& =  w_{k} - w^* - a_k * [\nabla g(w_k')( w_{k} - w^*)+\eta_k] \\
\\
\Delta_{k+1} & = \Delta_k - a_k[\nabla g(w_k')\Delta_k +\eta_k] \\
& = (1-a_k\nabla g(w_k'))\Delta_k+a_k(-\eta_k) \ , 取\alpha_k=1-a_k\nabla g(w_k') \\
& = \alpha_k\Delta_k+a_k(-\eta_k)
\end{align}$$
  - <font color=red>注意，在$ \tilde g(w_k,\ \eta_k)=g(w_k)+\eta_k$的分解中，所有的随机性都包含在$\eta_k$中</font>

#### I.6.3 Dvorotzky's theorem的一种扩展形式

- <font color=norange>在TD类算法的收敛性就是基于Dvorotzky's theorem的这种扩展形式。证明的要点就在于，虽然迭代过程中$V(s_k)需要来自V(s_{k+1})$的信息，但是$V(s_{k+1})$可以放到$\eta(s_k)_k$中将其视为对状态$s_k$的不确定性，只要整个$\eta(s_k)$满足条件，就不影响整个迭代的收敛性。</font>

- <img src='pics/dvorotzky_extension.png' width='70%'>

## II. RM算法的两大典型应用

1. 尽管算法本身是解决g(w)=0的问题，但$g(w)=E\tilde g(w, \eta)$，可见RM算法本质上是一种通过样本取值估计均值的问题。所以RM算法的第一类应用就是估计$Ef(x)$

2. 根据前文所述，凸优化与求根问题之间有等价关系。因此，算法的第类典型应用就是求解优化问题。此时，得到的迭代式结果与SGD算法一致。
   - 当$J(\theta)=Ef(x;\theta)$时:
$$\begin{align}
\underset{\theta }{min} J(\theta) =\underset{\theta }{min} E_xf(\theta, x) & \Leftrightarrow \nabla_\theta E_xf(\theta, x)  = 0 \\
& \Leftrightarrow E_x\nabla_\theta f(\theta, x)  = 0 \\
\end{align}$$

### II.1 RM算法求均值

#### 1. RM算法求解函数均值

- 原问题：已知样本$\{(x_i, f(x_i))\}_{i=1}^N$，求$Ef(X)=w$
- 转化为RM问题：$g(w)=w-Ef(X)=0$<font color=red> [注：不能是$g(w)=Ef(X)-w=0$，否则不满足RM算法的第一个条件]</font>

- 构造迭代要素：
$$
\begin{align} 
\tilde g(w, \eta) & = w-f(x) \\
\eta & = \tilde g(w, \eta) - g(w) \\
& = [w-f(x)]-[w-Ef(X)] \\
& = Ef(X)-f(x)
\end{align}
$$

- 得到迭代式
$$
\begin{align} 
w_{k+1} 
& = w_k - \alpha_k\tilde g(w_k, \eta) \\
& = w_k - \alpha_k(w_k-f(x_k)) \\
本质：\\
新估计 & = 旧估计 - 学习速度*[旧估计 - 新样本值]
\end{align}
$$

#### 2. RM算法求样本均值

- 当取f(x)=x时，上式等价于均值估计：
$$w_{k+1} = w_{k}-a_k(w_k-x_k)$$
$$取a_k=\frac{1}{k}，w_{k+1} = w_{k}-\frac{1}{k}(w_k-x_k)$$

### II.2 RM算法求函数最小值：SGD

#### II.2.1 用RM算法推导SGD迭代式

1. **RM算法求解函数期望值的最小化问题**： $\underset{w}{min}Ef(w, x)$
   - 原问题：$\underset{\theta}{min} J(\theta)  =\underset{\theta}{min}  E[f(\theta, X)]$
   - 等价问题：$E_x\nabla_\theta f(\theta, x)  = 0$
   - 转化为RM算法可解问题：$g(\theta) = E[\nabla_{\theta}f(\theta, X)]$
   - <font color=blue>**用RM算法得到和SGD相同的迭代形式**</font>：
$$
\begin{align}
g(\theta) & = E[\nabla_{\theta}f(\theta, x)]\\
\tilde{g}(\theta,\eta ) &= \nabla_{\theta}f(\theta, x) \\
\eta &= \tilde{g}(\theta,\eta ) - g(\theta)\\&= \nabla_{\theta}f(\theta, x)-E[\nabla_{\theta}f(\theta, x)]\\
\\
迭代形式：
\theta_{k+1} & = \theta_k -a_k*\nabla_{\theta}f(\theta_k, x_k)
\end{align}
$$
   - <font color=brown>[符号说明：$\theta和w$表示相同含义]</font>

2. <font color=green>**RM算法也可以推导解决样本均值最小化问题(非期望形态)的SGD**：</font>
   - 原问题：$\underset{w}{min} \frac{1}{n}\sum_i f(w, x_i)$
   - 解决思路：将deterministic value的n个x取值集合${x_1, x_2, ..., x_n}$转化为分布$P(X=x_i)=\frac{1}{n}$。此时有：
$$\frac{1}{n}\sum_i f(w, x_i)=Ef(w, x)$$
   - 转化为等价的期望值最小化问题：$\underset{w}{min} \frac{1}{n}\sum_i f(w, x_i)=\underset{w}{min}Ef(w, x)$

#### II.2.2 SGD算法的收敛性质

1. SGD的收敛条件：<font color=green>**对应RM算法的3个条件**</font>
   - (1) $0<c_1\le \nabla ^2 _wf(w,X)\le c_2$。自然有$0<c_1\le E[\nabla ^2 _wf(w,X)]\le c_2$。且此时f(w,X)相对w是凸函数。
   - (2) $\sum^{\infty}_{k=1}a_k=\infty,且\sum^{\infty}_{k=1}a_k^2<\infty$
   - (3) $\{x_k\}_{k=1}^{\infty}是iid的$。此时可以证明$E\eta=0$，用$|\nabla f(w_k,x)|<\infty$可以证明$E\eta^2<\infty$
$$
\begin{align}
E\eta & = E\left (  \nabla_wf(w, x)-E[\nabla_wf(w, X)]  \right )\\
& = E[\nabla_wf(w, x)]-E[\nabla_wf(w, X)]，\because \{x_i\}是iid的 \\
& = 0\\
\end{align}
$$
   - 满足这3个条件后，$w_k$almost surely converge to $\nabla_wEf(w, X)=0$的解。<font color=brown>[证明见math of RM的6.4.5节]</font>

2. 定义一个用于衡量收敛结果的指标：relative error
$$\delta_k = \frac{\left | \nabla_wf(w_k, x_k)-E[\nabla_wf(w_k, X)] \right | }{\left | E[\nabla_wf(w_k, X)] \right | } $$
   - **含义**：<font color=blue>分子是stochastic gradient$\nabla_wf(w_k, x_k)$与true gradient$E[\nabla_wf(w_k, X)]$的距离，整个式子表示stochastic gradient与true gradient的相对距离。</font>

3. relative error有性质：
$$
\delta_k \le \frac{\left | \nabla_wf(w_k, x_k)-E[\nabla_wf(w_k, X)] \right | }{c\left | w_k-w^* \right | }
$$

<font color=grey>
- 证明：
$$
\begin{align}
\because  E[\nabla_wf(w^*, X)]& =0 \\
\therefore  E[\nabla_wf(w_k, X)] & = E[\nabla_wf(w_k, X)]-E[\nabla_wf(w^*, X)] \\
 根据中值定理：& \exists \tilde w_k \in [w_k,w^*]，使得：\\
\left | E[\nabla_wf(w_k, X)] \right |  & = \left |E[\nabla_wf(w_k, X)]-E[\nabla_wf(w^*, X)] \right |\\
& =\left |(w_k-w^*)E[\nabla^2_wf(\tilde w_k, X)]\right | \\
& \ge c\left |w_k-w^*\right |，\because 当SGD满足收敛条件，就有\nabla^2_wf(w, X) \ge c_1 > 0  \\
代入上面的定义即可得证
\end{align}
$$</font>

4. **SGD的convergence pattern**
   - 上述不等式的分母代表了$w_k$与最优值$w_*$的距离，relative error与之成反比。**结论：<font color=green>当$w_k$距离最优值很远的时候，分母大，relative error很小，所以stochastic gradient与true gradient的差异就很小。这时候使用SGD与BGD的效果接近。反之，当$w_k$逼近真实值之后，分母变大，这时候relative error很可能较大，SGD在这时候与BGD的差异很大，此时convergence轨迹会出现更多randomness。</font>**